In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2232
1584
2448
2232
2304


In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 1512]
[2304 2232]
[2232 2448]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 1512]
[2232 2448]
[2232 2304]


In [12]:
images_ds = tf.data.Dataset.list_files('image/*/*', shuffle=False)

In [13]:
image_count = len(images_ds)
image_count

10

In [14]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'image\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'image\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'image\\cat\\A cat appears to have caught the....jpg'


In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'image\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg'
b'image\\dog\\100_ Dogs Pictures _ Download Free....jpg'
b'image\\dog\\11 Things Humans Do That Dogs Hate.jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

8

In [20]:
len(test_ds)

2

In [21]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]


In [22]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [23]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label


In [24]:
img, label = process_image("image\\dog\\11 Things Humans Do That Dogs Hate.jpg")
img.numpy()[:2]

array([[[131.81079 , 107.81079 ,  97.81079 ],
        [132.91205 , 105.91205 ,  96.91205 ],
        [134.3471  , 107.34711 ,  98.34711 ],
        [133.84271 , 106.84271 ,  97.84271 ],
        [133.24951 , 106.24951 ,  97.24951 ],
        [138.36444 , 111.36444 , 102.36444 ],
        [132.84344 , 105.843445,  96.843445],
        [143.3393  , 116.339294, 107.339294],
        [136.69531 , 109.69531 , 100.69531 ],
        [129.84088 , 102.84088 ,  93.84088 ],
        [137.34375 , 110.34375 , 101.34375 ],
        [133.6416  , 106.6416  ,  97.6416  ],
        [130.96576 , 103.96576 ,  94.96576 ],
        [135.06854 , 108.06854 ,  99.06854 ],
        [141.67316 , 114.67316 , 105.67316 ],
        [133.06427 , 106.06427 ,  95.06427 ],
        [134.32855 , 107.32855 ,  96.32855 ],
        [119.953064,  92.953064,  83.953064],
        [104.63281 ,  77.63281 ,  68.63281 ],
        [ 87.35663 ,  62.35663 ,  55.35663 ],
        [ 97.23425 ,  72.23425 ,  65.23425 ],
        [ 93.727234,  68.727234,  

In [25]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [26]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 24.         19.         15.       ]
  [ 24.         19.         15.       ]
  [ 21.863281   18.863281   13.863281 ]
  ...
  [143.92188   142.92188   148.92188  ]
  [153.92847   151.08228   158.00537  ]
  [104.69971    99.51221    97.07471  ]]

 [[ 22.076904   17.076904   13.076904 ]
  [ 24.410156   19.410156   15.410156 ]
  [ 18.07373    15.07373    10.07373  ]
  ...
  [144.7373    143.7373    149.7373   ]
  [150.03638   154.59888   157.59888  ]
  [123.66211   117.66211   120.03711  ]]

 [[ 22.1875     17.1875     13.1875   ]
  [ 21.615479   16.615479   12.6154785]
  [ 20.         17.         12.       ]
  ...
  [147.12085   146.12085   152.12085  ]
  [147.23486   148.23486   152.23486  ]
  [152.54517   150.54517   155.031    ]]

 ...

 [[138.17798   143.17798   146.17798  ]
  [144.43701   149.1206    152.1206   ]
  [163.23169   169.18091   172.8645   ]
  ...
  [128.80176   132.43457   137.11816  ]
  [153.0188    150.84082   161.84082  ]
  [100.593994  102.22681   1

In [27]:
def scale(image, label):
    return image/255, label

In [28]:
train_ds = train_ds.map(scale)

In [31]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0][0])
    print("****Label: ",label.numpy())

****Image:  0.7549632
****Label:  b'dog'
****Image:  0.7736826
****Label:  b'cat'
****Image:  0.6878983
****Label:  b'cat'
****Image:  0.039246324
****Label:  b'cat'
****Image:  0.7009957
****Label:  b'dog'
